In [1]:
import pandas as pd
import os
import datetime
import difflib
from google.cloud import bigquery
from google.oauth2 import service_account
import numpy as np
import time

In [2]:
def get_latest_file2(directory):
    files = [f for f in os.listdir(directory) 
             if f.endswith('.XLSX') and not f.startswith('~$') and f.startswith('Query')]
    if not files:
        raise FileNotFoundError(f"No Excel files found in {directory}")
    files = [os.path.join(directory, f) for f in files]
    latest_file = max(files, key=os.path.getmtime)
    print(f'Lendo {latest_file}')
    return latest_file

def inverter_data(data):
    if pd.isna(data):
        return np.nan
    
    if isinstance(data, str):
        if '/' in data:
            dia, mes, ano = data.split('/')
            return ano + mes + dia
        elif len(data) >= 10 and data[4] == '-' and data[7] == '-':
            return data[:10].replace('-', '')  # Converte 'yyyy-mm-dd' para 'yyyymmdd'
    
    return np.nan

In [ ]:
credenciais_json = {
}

credentials = service_account.Credentials.from_service_account_info(credenciais_json)
client = bigquery.Client(credentials=credentials, project=credenciais_json["project_id"])

In [4]:
caminho_siglas = r'\\grupofleury\Auditoria\Auditoria_Interna\Auditoria_Continua\Rogerio.Mello\TRABALHOS REALIZADOS\2 - [ADITIVOS] Leitura de PDFs e Imagens, Comparativos, Criação de Interface\2.10 - OPAs sigla contratada'
data_atual = datetime.datetime.now()
data_formatada = data_atual.strftime("%d-%m-%Y")
data_hoje = pd.to_datetime(datetime.datetime.now().date()).strftime('%d/%m/%Y')
data_hoje = int(inverter_data(data_hoje))


In [5]:
siglas_df = pd.read_excel(caminho_siglas + r'\Siglas.xlsx', sheet_name='Sheet1')

In [6]:
siglas_df

,Siglas Fleury,Descrição
0,MUTBRAF,"Gene BRAF, Detecção da mutação V600E, fragment..."
1,EGFR,"Detecção de mutações no gene EGFR, biópsia"
2,MUTPRO,"Protrombina e Fator V Leiden, Mutação do gene,..."
3,LEIDEN,"Fator V de Leiden, Mutação do gene, pesquisa, ..."
4,FISHHER,"Her 2, Hibridação in situ por fluorescência"
...,...,...
79,PH1PCR,"BCR-ABL, Tipo P190 e P210, quanti. por RT-PCR ..."
80,QBCRABL,"BCR-ABL tipo P210, quantificação por tempo rea..."
81,MUTJAK2,"Mutação V617F no gene JAK2, Detecção quantitat..."
82,CALRET,"Pesquisa de mutações no exon 9, do gene da Cal..."


In [7]:
# Converter siglas do DataFrame para uma string formatada para SQL
siglas = "', '".join(siglas_df['Siglas Fleury'].astype(str).unique())

# Criar a query SQL
query = f"""
SELECT DISTINCT TABELA_PRECO, SIGLA_PRODUTO 
FROM `data-lake-prd-276215.rz.BI_RZTB_TABELAS_PRECO_FINAL_PRICING` 
WHERE SIGLA_PRODUTO IN ('{siglas}');
"""

print("query")
requisicao = client.query(query)
print("to dataframe")
df5 = requisicao.result().to_dataframe()

query
to dataframe


c:\Users\rogerio.mello\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [8]:
df5

,TABELA_PRECO,SIGLA_PRODUTO
0,OMINT-co-V_15,LEUGERMINATIVO
1,NEOLAB-FL,POLIPOSEHD
2,VIVESCESP-FMF4,CANCERURO
3,SEISA-ASP-V7,POLIPOSEHD
4,LABDOLES-FL-V25,PANCREASHD
...,...,...
158911,CMD-FL-V25,XFRAGILPCR
158912,INSTDADOR-APEV1,XFRAGILPCR
158913,IMUNOCORE-FL1,XFRAGILPCR
158914,LABAPOLO,XFRAGILPCR


In [13]:
Query_sap_real_dir = fr'\\grupofleury\Dir_Exec_Suporte_Operacoes\GCC - Query SAP\2025\03 Março _2025'
Query_sap_real = get_latest_file2(Query_sap_real_dir)
df_sap = pd.read_excel(Query_sap_real, usecols=['Nome da sigla de contrato', 'Descrição Convênio', 'Descrição Plano', 'Tabela de Preços', 'Validade do plano até', 'Válido até', 'Descrição Empresa', 'Capitation'], keep_default_na=False, na_values=[])


###### TRATAMENTO QUERY_SAP
df_sap[['Validade do plano até', 'Válido até']] = (df_sap[['Validade do plano até', 'Válido até']].fillna('').astype(str))
df_sap['Validade do plano até'] = df_sap['Validade do plano até'].apply(inverter_data)
df_sap['Válido até'] = df_sap['Válido até'].apply(inverter_data)
df_sap[['Validade do plano até', 'Válido até']] = df_sap[['Validade do plano até', 'Válido até']].fillna(0).astype(int)
df_sap= df_sap[df_sap['Validade do plano até'] > data_hoje]
df_sap= df_sap[df_sap['Válido até'] > data_hoje]
df_sap = df_sap[df_sap['Capitation'] != 'S']


Lendo \\grupofleury\Dir_Exec_Suporte_Operacoes\GCC - Query SAP\2025\03 Março _2025\QuerySap 10_03_2025.XLSX


In [16]:
ativos = pd.merge(df5, df_sap, left_on='TABELA_PRECO', right_on='Tabela de Preços', how='left')

In [17]:
ativos.to_excel(caminho_siglas + fr'\Ativos_{data_formatada}.xlsx', index=False)

ValueError: This sheet is too large! Your sheet size is: 1082904, 10 Max sheet size is: 1048576, 16384

In [19]:
grupos = ativos.groupby("SIGLA_PRODUTO")
dfs = []
df_temp = pd.DataFrame()

limite_excel = 1048576

for _, grupo in grupos:
    if len(df_temp) + len(grupo) > limite_excel:
        dfs.append(df_temp)  # Salva o DataFrame atual
        df_temp = pd.DataFrame()  # Reinicia para um novo arquivo
    df_temp = pd.concat([df_temp, grupo])

# Adiciona o último DataFrame se não estiver vazio
if not df_temp.empty:
    dfs.append(df_temp)

In [20]:

# Salvar os arquivos
for i, df_parte in enumerate(dfs):
    df_parte.to_excel(caminho_siglas + fr'\Ativos_{data_formatada} {i+1}.xlsx', index=False, sheet_name="Dados")

In [18]:
ativos

,TABELA_PRECO,SIGLA_PRODUTO,Descrição Convênio,Descrição Empresa,Descrição Plano,Nome da sigla de contrato,Tabela de Preços,Validade do plano até,Válido até,Capitation
0,OMINT-co-V_15,LEUGERMINATIVO,Labs amais - Omint,Omint Assistencial Serviços de Saúde,Access Medicina Completo-prata,OMINT-LAB-F,OMINT-co-V_15,99991231.0,99991231.0,
1,OMINT-co-V_15,LEUGERMINATIVO,Labs amais - Omint,Omint Assistencial Serviços de Saúde,Access Prata - sem valid,OMINTSV-LAB-F,OMINT-co-V_15,99991231.0,99991231.0,
2,OMINT-co-V_15,LEUGERMINATIVO,Labs amais - Omint,Omint Assistencial Serviços de Saúde,Atendimento Excepcional,OMINTAE-LAB-F,OMINT-co-V_15,99991231.0,99991231.0,
3,OMINT-co-V_15,LEUGERMINATIVO,Labs amais - Omint,Omint Assistencial Serviços de Saúde,C Azul L Douradas - sem valid,OMINTSV-LAB-F,OMINT-co-V_15,99991231.0,99991231.0,
4,OMINT-co-V_15,LEUGERMINATIVO,Labs amais - Omint,Omint Assistencial Serviços de Saúde,C Verde L Douradas - sem valid,OMINTSV-LAB-F,OMINT-co-V_15,99991231.0,99991231.0,
...,...,...,...,...,...,...,...,...,...,...
1082899,CMD-FL-V25,XFRAGILPCR,LABORATÓRIOS CLIENTES,CMD-FL CMD - Centro de Med. Diag. Ltda.,Todos,CMD-FL,CMD-FL-V25,99981231.0,99991231.0,
1082900,INSTDADOR-APEV1,XFRAGILPCR,amais PE - Instituto da dor,Clínica da Dor,Todos - à vista,INSTDADOR-APE-V,INSTDADOR-APEV1,99991231.0,99991231.0,
1082901,IMUNOCORE-FL1,XFRAGILPCR,LABORATÓRIOS CLIENTES,IMUNOCORE-FL Imunocore lab. patologia,Todos,IMUNOCORE-FL,IMUNOCORE-FL1,99991231.0,99991231.0,
1082902,LABAPOLO,XFRAGILPCR,LABORATÓRIOS CLIENTES,LABAPOLO-FL Lab de analises Apolo Ltda,Todos,LABAPOLO-FL,LABAPOLO,99991231.0,99991231.0,N
